In [1]:
library(Seurat)
library(SeuratDisk)

Attaching SeuratObject

Registered S3 method overwritten by 'cli':
  method     from         
  print.boxx spatstat.geom

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat



In [13]:
fmap = list(
    list(
        list('/data/pollock/benchmarking/pollock_datasets/scRNAseq/brca_train.rds',
             '/data/pollock/benchmarking/pollock_datasets/scRNAseq/brca_val.rds'),
        list('/data/pollock/benchmarking/pollock_datasets/scRNAseq/cesc_train.rds',
             '/data/pollock/benchmarking/pollock_datasets/scRNAseq/cesc_val.rds'),
        list('/data/pollock/benchmarking/pollock_datasets/scRNAseq/hnscc_train.rds',
             '/data/pollock/benchmarking/pollock_datasets/scRNAseq/hnscc_val.rds'),
        list('/data/pollock/benchmarking/pollock_datasets/scRNAseq/melanoma_train.rds',
             '/data/pollock/benchmarking/pollock_datasets/scRNAseq/melanoma_val.rds'),
        list('/data/pollock/benchmarking/pollock_datasets/scRNAseq/myeloma_train.rds',
             '/data/pollock/benchmarking/pollock_datasets/scRNAseq/myeloma_val.rds'),
        list('/data/pollock/benchmarking/pollock_datasets/scRNAseq/pdac_train.rds',
             '/data/pollock/benchmarking/pollock_datasets/scRNAseq/pdac_val.rds')
    ),
    list(
        list('/data/pollock/benchmarking/pollock_datasets/snRNAseq/brca_train.rds',
             '/data/pollock/benchmarking/pollock_datasets/snRNAseq/brca_val.rds'),
        list('/data/pollock/benchmarking/pollock_datasets/snRNAseq/ccrcc_train.rds',
             '/data/pollock/benchmarking/pollock_datasets/snRNAseq/ccrcc_val.rds'),
        list('/data/pollock/benchmarking/pollock_datasets/snRNAseq/gbm_train.rds',
             '/data/pollock/benchmarking/pollock_datasets/snRNAseq/gbm_val.rds')
    ),
    list(
        list('/data/pollock/benchmarking/pollock_datasets/snATACseq/brca_gene_activity_train.rds',
             '/data/pollock/benchmarking/pollock_datasets/snATACseq/brca_gene_activity_val.rds'),
        list('/data/pollock/benchmarking/pollock_datasets/snATACseq/ccrcc_gene_activity_train.rds',
             '/data/pollock/benchmarking/pollock_datasets/snATACseq/ccrcc_gene_activity_val.rds'),
        list('/data/pollock/benchmarking/pollock_datasets/snATACseq/gbm_gene_activity_train.rds',
             '/data/pollock/benchmarking/pollock_datasets/snATACseq/gbm_gene_activity_val.rds')
    )
)

In [2]:
predict_cell_types <- function(train_obj, val_obj) {
    obj.list = c(train_obj, val_obj)
    names(obj.list) = c('train', 'val')
    
    for (i in 1:length(obj.list)) {
        obj.list[[i]] <- NormalizeData(obj.list[[i]], verbose = FALSE)
        obj.list[[i]] <- FindVariableFeatures(obj.list[[i]], selection.method = "vst", nfeatures = 2000,
            verbose = FALSE)
    }
#     obj.anchors <- FindIntegrationAnchors(object.list = obj.list, dims = 1:30)
#     obj.integrated <- IntegrateData(anchorset = obj.anchors, dims = 1:30)
    
#     obj.integrated <- ScaleData(obj.integrated, verbose = FALSE)
#     obj.integrated <- RunPCA(obj.integrated, npcs = 30, verbose = FALSE)
#     obj.integrated <- RunUMAP(obj.integrated, reduction = "pca", dims = 1:30, verbose = FALSE)
    
    train_obj <- ScaleData(obj.list[["train"]], verbose = FALSE)
    train_obj <- RunPCA(train_obj, npcs = 30, verbose = FALSE)
    train_obj <- RunUMAP(train_obj, reduction = "pca", dims = 1:30, verbose = FALSE)
    val_obj <- obj.list[["val"]]
    
    obj.anchors <- FindTransferAnchors(reference = train_obj, query = val_obj,
        dims = 1:30, reference.reduction = "pca")
    predictions <- TransferData(anchorset = obj.anchors, refdata = train_obj$cell_type,
        dims = 1:30)
    val_obj <- AddMetaData(val_obj, metadata = predictions)
    
    return(val_obj)
}

In [3]:
saveH5ad <- function(obj, h5_fp) {
    obj <- DietSeurat(obj, assay = 'RNA')
    for (col in colnames(obj@meta.data)) {
        if (!is.null(as.numeric(as.character(obj[[col]][1])))) {
            obj@meta.data[[col]] = as.character(obj@meta.data[[col]])
        }
    }
    SaveH5Seurat(obj, filename = h5_fp, overwrite = T)
    Convert(h5_fp, dest = "h5ad", overwrite = T)
}

In [14]:
for (dtype_list in fmap) {
    for (disease_list in dtype_list) {
        train_fp = disease_list[[1]]
        val_fp = disease_list[[2]]
        
        if (!grepl('gene_activity', train_fp, fixed = TRUE)) {
            dtype = strsplit(strsplit(train_fp, "_train")[[1]][[1]], "/")[[1]][[6]]
            disease = strsplit(strsplit(train_fp, "_train")[[1]][[1]], "/")[[1]][[7]]
        } else {
            dtype = strsplit(strsplit(train_fp, "_gene_activity_train")[[1]][[1]], "/")[[1]][[6]]
            disease = strsplit(strsplit(train_fp, "_gene_activity_train")[[1]][[1]], "/")[[1]][[7]]
        }
        
        print(dtype)
        print(disease)
        
        train <- readRDS(train_fp)
        val <- readRDS(val_fp)
        
        train <- CreateSeuratObject(train@assays$RNA@counts, project = "train", assay = "RNA",
          min.cells = 0, min.features = 0, meta.data = train@meta.data)
        
        val <- CreateSeuratObject(val@assays$RNA@counts, project = "val", assay = "RNA",
          min.cells = 0, min.features = 0, meta.data = val@meta.data)
        
        
        pred_obj = predict_cell_types(train, val)
        
        h5_fp = paste("/data/pollock/benchmarking/results/seurat/", dtype, "_", disease, ".h5seurat", sep = "")
        saveH5ad(pred_obj, h5_fp)
    }
}

[1] "snATACseq"
[1] "brca"


Projecting cell embeddings

Finding neighborhoods

Finding anchors

	Found 5104 anchors

Filtering anchors

	Retained 2349 anchors

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(p

[1] "snATACseq"
[1] "ccrcc"


Projecting cell embeddings

Finding neighborhoods

Finding anchors

	Found 4780 anchors

Filtering anchors

	Retained 2227 anchors

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(p

[1] "snATACseq"
[1] "gbm"


Projecting cell embeddings

Finding neighborhoods

Finding anchors

	Found 4086 anchors

Filtering anchors

	Retained 2340 anchors

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(p

cross fold 

In [5]:
fmap = list(
    list(
        list(
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/brca_fold0_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/brca_fold0_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/brca_fold1_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/brca_fold1_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/brca_fold2_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/brca_fold2_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/brca_fold3_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/brca_fold3_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/brca_fold4_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/brca_fold4_val.rds')
        ),
        list(
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/cesc_fold0_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/cesc_fold0_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/cesc_fold1_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/cesc_fold1_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/brca_fold2_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/cesc_fold2_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/cesc_fold3_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/cesc_fold3_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/cesc_fold4_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/cesc_fold4_val.rds')
        ),
        list(
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/hnscc_fold0_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/hnscc_fold0_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/hnscc_fold1_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/hnscc_fold1_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/hnscc_fold2_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/hnscc_fold2_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/hnscc_fold3_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/hnscc_fold3_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/hnscc_fold4_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/hnscc_fold4_val.rds')
        ),
        list(
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/melanoma_fold0_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/melanoma_fold0_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/melanoma_fold1_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/melanoma_fold1_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/melanoma_fold2_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/melanoma_fold2_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/melanoma_fold3_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/melanoma_fold3_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/melanoma_fold4_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/melanoma_fold4_val.rds')
        ),
        list(
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/myeloma_fold0_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/myeloma_fold0_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/myeloma_fold1_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/myeloma_fold1_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/myeloma_fold2_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/myeloma_fold2_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/myeloma_fold3_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/myeloma_fold3_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/myeloma_fold4_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/myeloma_fold4_val.rds')
        ),
        list(
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/pdac_fold0_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/pdac_fold0_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/pdac_fold1_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/pdac_fold1_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/pdac_fold2_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/pdac_fold2_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/pdac_fold3_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/pdac_fold3_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/pdac_fold4_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/pdac_fold4_val.rds')
        )
    ),
    list(
        list(
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/brca_fold0_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/brca_fold0_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/brca_fold1_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/brca_fold1_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/brca_fold2_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/brca_fold2_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/brca_fold3_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/brca_fold3_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/brca_fold4_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/brca_fold4_val.rds')
        ),
        list(
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/ccrcc_fold0_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/ccrcc_fold0_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/ccrcc_fold1_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/ccrcc_fold1_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/ccrcc_fold2_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/ccrcc_fold2_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/ccrcc_fold3_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/ccrcc_fold3_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/ccrcc_fold4_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/ccrcc_fold4_val.rds')
        ),
        list(
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/gbm_fold0_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/gbm_fold0_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/gbm_fold1_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/gbm_fold1_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/gbm_fold2_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/gbm_fold2_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/gbm_fold3_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/gbm_fold3_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/gbm_fold4_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snRNAseq/gbm_fold4_val.rds')
        )
    ),
    list(
        list(
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/brca_fold0_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/brca_fold0_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/brca_fold1_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/brca_fold1_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/brca_fold2_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/brca_fold2_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/brca_fold3_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/brca_fold3_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/brca_fold4_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/brca_fold4_val.rds')
        ),
        list(
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/ccrcc_fold0_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/ccrcc_fold0_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/ccrcc_fold1_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/ccrcc_fold1_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/ccrcc_fold2_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/ccrcc_fold2_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/ccrcc_fold3_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/ccrcc_fold3_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/ccrcc_fold4_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/ccrcc_fold4_val.rds')
        ),
        list(
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/gbm_fold0_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/gbm_fold0_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/gbm_fold1_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/gbm_fold1_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/gbm_fold2_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/gbm_fold2_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/gbm_fold3_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/gbm_fold3_val.rds'),
            list('/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/gbm_fold4_train.rds',
             '/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/gbm_fold4_val.rds')
        )
    )
)

In [5]:
for (dtype_list in fmap) {
    for (d1 in dtype_list) {
        for (disease_list in d1) {
            train_fp = disease_list[[1]]
            val_fp = disease_list[[2]]

            dtype = strsplit(strsplit(train_fp, "_train")[[1]][[1]], "/")[[1]][[6]]
            disease = strsplit(strsplit(train_fp, "_train")[[1]][[1]], "/")[[1]][[7]]

            print(dtype)
            print(disease)

            train <- readRDS(train_fp)
            val <- readRDS(val_fp)

            train <- CreateSeuratObject(train@assays$RNA@counts, project = "train", assay = "RNA",
              min.cells = 0, min.features = 0, meta.data = train@meta.data)

            val <- CreateSeuratObject(val@assays$RNA@counts, project = "val", assay = "RNA",
              min.cells = 0, min.features = 0, meta.data = val@meta.data)


            pred_obj = predict_cell_types(train, val)

            h5_fp = paste("/data/pollock/benchmarking/results/seurat/interdataset_with_folds/", dtype, "_", disease, ".h5seurat", sep = "")
            saveH5ad(pred_obj, h5_fp)
        }
    }
}

[1] "scRNAseq"
[1] "brca_fold0"


Warning message:
“The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session”
Projecting cell embeddings

Finding neighborhoods

Finding anchors

	Found 12729 anchors

Filtering anchors

	Retained 11353 anchors

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_

[1] "scRNAseq"
[1] "brca_fold1"


Projecting cell embeddings

Finding neighborhoods

Finding anchors

	Found 12929 anchors

Filtering anchors

	Retained 11717 anchors

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad

[1] "scRNAseq"
[1] "brca_fold2"


...

Projecting cell embeddings

Finding neighborhoods

Finding anchors

	Found 5040 anchors

Filtering anchors

	Retained 3321 anchors

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(p

[1] "snATACseq"
[1] "gbm_fold3"


Projecting cell embeddings

Finding neighborhoods

Finding anchors

	Found 4016 anchors

Filtering anchors

	Retained 2273 anchors

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(p

[1] "snATACseq"
[1] "gbm_fold4"


Projecting cell embeddings

Finding neighborhoods

Finding anchors

	Found 4656 anchors

Filtering anchors

	Retained 2813 anchors

Finding integration vectors

Finding integration vector weights

Predicting cell labels

Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(p

In [6]:
strsplit('/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/gbm_fold0_train.rds', "_train")[[1]][[1]]

[1] "/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/gbm_fold0"

In [7]:
f = '/data/pollock/benchmarking/pollock_datasets_with_folds/snATACseq/gbm_fold0_train.rds'
strsplit(strsplit(f, "_train")[[1]][[1]], "/")[[1]][[6]]

[1] "snATACseq"

In [8]:
strsplit(strsplit(f, "_train")[[1]][[1]], "/")[[1]][[7]]

[1] "gbm_fold0"

In [6]:
train_fp = '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/brca_fold2_train.rds'
val_fp = '/data/pollock/benchmarking/pollock_datasets_with_folds/scRNAseq/brca_fold2_val.rds'

dtype = strsplit(strsplit(train_fp, "_train")[[1]][[1]], "/")[[1]][[6]]
disease = strsplit(strsplit(train_fp, "_train")[[1]][[1]], "/")[[1]][[7]]

print(dtype)
print(disease)

train <- readRDS(train_fp)
val <- readRDS(val_fp)

[1] "scRNAseq"
[1] "brca_fold2"


In [7]:
train <- CreateSeuratObject(train@assays$RNA@counts, project = "train", assay = "RNA",
  min.cells = 0, min.features = 0, meta.data = train@meta.data)

val <- CreateSeuratObject(val@assays$RNA@counts, project = "val", assay = "RNA",
  min.cells = 0, min.features = 0, meta.data = val@meta.data)

In [9]:
val@meta.data

,orig.ident,nCount_RNA,nFeature_RNA,cell_type,barcode,sample
,<fct>,<dbl>,<int>,<fct>,<fct>,<int>
29_TTCTTCCGTACGATGG-1,29,3410,1293,CD8 T cell,TTCTTCCGTACGATGG-1,29
1_GTGTAACCACCTGCAG-1,1,3438,917,CD8 T cell,GTGTAACCACCTGCAG-1,1
17_TCGTAGACAGCTGAAG-1,17,4369,1884,CD8 T cell,TCGTAGACAGCTGAAG-1,17
7_AGAAATGCACCCATAA-1,7,3264,1100,CD8 T cell,AGAAATGCACCCATAA-1,7
15_TTTACTGGTTTGAACC-1,15,3316,923,CD8 T cell,TTTACTGGTTTGAACC-1,15
26_TTGTTCACACGAGGTA-1,26,2855,723,CD8 T cell,TTGTTCACACGAGGTA-1,26
28_ATATCCTCAGGGCTTC-1,28,3338,1368,CD8 T cell,ATATCCTCAGGGCTTC-1,28
0_CAGGTATCAGAGTTCT-1,0,4177,1828,CD8 T cell,CAGGTATCAGAGTTCT-1,0
27_CTTCCTTAGCTTGTTG-1,27,3689,1459,CD8 T cell,CTTCCTTAGCTTGTTG-1,27


In [10]:


pred_obj = predict_cell_types(train, val)


Warning message:
“The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session”
Projecting cell embeddings

Finding neighborhoods

Finding anchors

	Found 12888 anchors

Filtering anchors

	Retained 11649 anchors

Finding integration vectors

Finding integration vector weights

Predicting cell labels



In [12]:
head(pred_obj@meta.data)

,orig.ident,nCount_RNA,nFeature_RNA,cell_type,barcode,sample,predicted.id,prediction.score.CD8.T.cell,prediction.score.Endothelial,prediction.score.Fibroblast,⋯,prediction.score.NK,prediction.score.Monocyte,prediction.score.Treg,prediction.score.CD4.T.cell,prediction.score.B.cell,prediction.score.Plasma,prediction.score.Mast,prediction.score.Dendritic,prediction.score.Erythrocyte,prediction.score.max
,<fct>,<dbl>,<int>,<fct>,<fct>,<int>,<chr>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
29_TTCTTCCGTACGATGG-1,29,3410,1293,CD8 T cell,TTCTTCCGTACGATGG-1,29,CD8 T cell,0.9021679,0,0,⋯,0.08611380,0,0.00000000,0.00000000,0,0.011718273,0,0,0,0.9021679
1_GTGTAACCACCTGCAG-1,1,3438,917,CD8 T cell,GTGTAACCACCTGCAG-1,1,CD8 T cell,0.7297186,0,0,⋯,0.06354818,0,0.02975257,0.17698066,0,0.000000000,0,0,0,0.7297186
17_TCGTAGACAGCTGAAG-1,17,4369,1884,CD8 T cell,TCGTAGACAGCTGAAG-1,17,CD8 T cell,0.9103750,0,0,⋯,0.01642992,0,0.00000000,0.06368967,0,0.009505392,0,0,0,0.9103750
7_AGAAATGCACCCATAA-1,7,3264,1100,CD8 T cell,AGAAATGCACCCATAA-1,7,CD8 T cell,0.8525243,0,0,⋯,0.07483228,0,0.01809246,0.05455097,0,0.000000000,0,0,0,0.8525243
15_TTTACTGGTTTGAACC-1,15,3316,923,CD8 T cell,TTTACTGGTTTGAACC-1,15,CD8 T cell,0.6575688,0,0,⋯,0.27172943,0,0.00000000,0.07070176,0,0.000000000,0,0,0,0.6575688
26_TTGTTCACACGAGGTA-1,26,2855,723,CD8 T cell,TTGTTCACACGAGGTA-1,26,CD4 T cell,0.2510602,0,0,⋯,0.00000000,0,0.13063138,0.61830842,0,0.000000000,0,0,0,0.6183084


In [13]:

h5_fp = paste("/data/pollock/benchmarking/results/seurat/interdataset_with_folds/", dtype, "_", disease, ".h5seurat", sep = "")
saveH5ad(pred_obj, h5_fp)

Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5ad(pred_obj, h5_fp):
“NAs introduced by coercion”
Warning message in saveH5